In [1]:
import autograd
from autograd import numpy as np
import scipy as sp
import paragami
from paragami import autograd_supplement_lib

from autograd.test_util import check_grads
from numpy.testing import assert_array_almost_equal


In [2]:
n_groups = 10
n_per_group = 5
n_obs = n_groups * n_per_group

x = np.random.random(n_obs)
groups = np.repeat(np.arange(0, n_groups), n_per_group)

In [3]:
bincount_result = np.bincount(groups, x)

In [4]:
# cols = np.arange(0, n_obs)
# data = np.ones(n_obs)
# rows = groups
# grouping_mat = sp.sparse.csr_matrix((data, (rows, cols)), (np.max(groups) + 1, n_obs))
# assert_array_almost_equal(grouping_mat @ x, bincount_result)

In [5]:
# get_grouped_sum, _ = autograd_supplement_lib.get_sparse_product(grouping_mat) 
# assert_array_almost_equal(get_grouped_sum(x), bincount_result)

aggregate = autograd_supplement_lib.get_grouped_aggregator(groups)

In [7]:
check_grads(aggregate)(x)

In [9]:
x2 = np.random.random((n_obs, 3))
check_grads(aggregate)(x2)

In [10]:
groups = np.atleast_1d(groups).astype('int64')
groups.dtype


dtype('int64')